In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

C:\Users\seolbluewings\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


#### AirQuality dataset
1. Solar.R 결측값이 존재하는 행 제거
2. Ozone 항목 결측값을 중앙값으로 대체
3. 중앙 값으로 대체하기 이전,이후의 Ozone 표준편차 차이 구하기

In [2]:
url = "https://assets.datacamp.com/production/course_2023/datasets/airquality.csv"
airquality = pd.read_csv(url)

#### 특정 컬럼에 결측값 있는 행 제거
- axis = 0 : 행 제거 / axis = 1 : 열 제거

In [3]:
airquality_v1 = airquality.dropna(subset = ['Solar.R'],axis = 0)
airquality_v2 = airquality_v1

In [4]:
print(len(airquality), len(airquality_v1))

153 146


In [5]:
airquality_v1.isnull().sum()

Ozone      35
Solar.R     0
Wind        0
Temp        0
Month       0
Day         0
dtype: int64

In [6]:
airquality_v1['Ozone'].median(skipna = True)

31.0

#### Ozone 컬럼의 결측값을 중앙 값으로 대체

In [7]:
airquality_v2['Ozone'] = airquality_v2['Ozone'].fillna(airquality_v1['Ozone'].median(skipna = True))

C:\Users\seolbluewings\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
airquality_v2.isnull().sum()

Ozone      0
Solar.R    0
Wind       0
Temp       0
Month      0
Day        0
dtype: int64

#### 컬럼의 표준편차 차이 계산

In [9]:
print(airquality_v2['Ozone'].std() - airquality_v1['Ozone'].std())

0.0


#### Hitter 데이터셋에서 Salary의 Outlier 합 구하기
- 중위수에서 IQR 2배 초과하는 값이 Outlier인 것으로 정의

In [3]:
Hitters = pd.read_csv(r'C:\Users\seolbluewings\Desktop\sample\Hitters.csv')

In [4]:
Hitters.head(5)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [12]:
IQR = Hitters['Salary'].quantile(q=0.75) - Hitters['Salary'].quantile(q=0.25)

- 중위수 - 2*IQR or 중위수 + 2*IQR 밖 범위에 있는 데이터를 추출
- 추출 과정에서 조건문에는 괄호() 처리해주어야 함 (중요)

In [13]:
Outlier = Hitters[(Hitters['Salary'] < Hitters['Salary'].median()-2*IQR) | (Hitters['Salary'] > Hitters['Salary'].median()+2*IQR)]

In [15]:
Outlier['Salary'].sum()

21671.864

#### diamonds 데이터 활용
- 순서대로 80% 데이터를 훈련 데이터로 만들기
- Price 기준으로 상위 100개 데이터에 대한 Price 평균 구하기

In [24]:
diamonds = pd.read_csv(r'C:\Users\seolbluewings\Desktop\sample\diamonds.csv')

In [25]:
diamonds = diamonds.iloc[:,1:10]

In [26]:
diamonds.head(5)

,carat,cut,color,clarity,depth,table,price,x,y
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35


In [19]:
nrow = int(len(diamonds)*0.8)

- 데이터 순서대로 80% 데이터 Parsing 

In [20]:
tr_set = diamonds.iloc[0:nrow,:]

- Price 기준으로 데이터 정렬 밑 상위 100개 데이터 추출하여 평균 반환
- 순서대로 정렬 sort_values 함수 사용, by = 정렬기준, ascending = True 오름차순, ascending = False 내림차순 

In [21]:
tr_set.sort_values(by = ['price'], ascending = False).iloc[0:100,6].mean()

18663.33

#### airquality 
- 데이터 순서대로 90% 추출하여 훈련 데이터로 설정
- Ozone 항목의 결측값을 평균으로 변경
- 변경 전,후의 중앙값 차이 비교

In [42]:
air_tr = airquality.iloc[0:int(round(len(airquality)*0.9,0)),:]
median_bf = air_tr['Ozone'].median(skipna = True)
air_tr['Ozone'] = air_tr['Ozone'].fillna(air_tr['Ozone'].mean(skipna = True))
median_af = air_tr['Ozone'].median(skipna= True)
print(median_af-median_bf)

10.049019607843135


C:\Users\seolbluewings\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### Hitters
- CHits 컬럼의 상위 25%, 하위 25%를 0으로 대체하여 별도 데이터셋 생성
- CHits 항목의 평균과 표준편차 합을 구하기

In [70]:
Hitters['CHits'][(Hitters['CHits']>=Hitters['CHits'].quantile(q = 0.75))|(Hitters['CHits']<=Hitters['CHits'].quantile(q = 0.25))] = 0
print(Hitters['CHits'].mean(skipna = True) + Hitters['CHits'].std(skipna = True))

597.8164297529598


C:\Users\seolbluewings\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


- HmRun 데이터 컬럼에서 이상값을 제외한 평균을 구하기
- 이상값은 평균에서 1.5 표준편차 이상인 값

In [73]:
IQR = Hitters['HmRun'].quantile(q=0.75) - Hitters['HmRun'].quantile(q=0.25)
Hitters_tr = Hitters[(Hitters['HmRun']<= Hitters['HmRun'].mean(skipna = True)+1.5*IQR)|(Hitters['HmRun']>= Hitters['HmRun'].mean(skipna = True)-1.5*IQR)]
Hitters_tr['HmRun'].mean(skipna = True)

10.770186335403727

- CHmRun 과 CRuns 의 피어슨 상관계수 구하기
- CHmRun이 10 이상인 데이터만 한정해서

In [78]:
Hitters_new = Hitters[Hitters['CHmRun']>=10]
Hitters_new[['CHmRun','CRuns']].corr(method = 'pearson')

,CHmRun,CRuns
CHmRun,1.000000,0.789996
CRuns,0.789996,1.000000


#### mtcars 데이터셋
- am = 1 에서 wt 가 작게 나가는 10개 데이터의 mpg 평균
- am = 0 에서 wt 가 작게 나가는 10개 데이터의 mpg 평균

In [39]:
df = pd.read_csv("C://Users/seolbluewings/Desktop/sample/mtcars.csv")

In [23]:
df.loc[df['am']==0,["mpg","wt"]].sort_values(by = "wt", ascending = True).head(10).mean()['mpg'] - df.loc[df['am']==1,["mpg","wt"]].sort_values(by = "wt", ascending = True).head(10).mean()['mpg']

-7.07

- 4기통(cyl = 4)인 자동차의 백분율을 구하기

In [40]:
mtcars = pd.read_csv("C://Users/seolbluewings/Desktop/sample/mtcars.csv")

In [46]:
mtcars.loc[df['cyl'] == 4,"am"].count() / len(mtcars)

0.34375

- 변속기어수(gear) 가 4이고 수동 변속기(am) 1 인 데이터에서
- 자동차 연비(mpg)의 mean 값과 전체 마력(hp)의 표준편차 합계 구하기

In [48]:
mtcars.loc[(mtcars['gear'] == 4) & (mtcars['am'] == 1),"mpg"].mean() + mtcars.loc[(mtcars['gear'] == 4) & (mtcars['am'] == 1),"hp"].std()

50.44958819268106

#### Diamonds 데이터
- 순서대로 80% 를 제거하고 
- cut 이 Fair 이면서 carat이 1 이상인 diamonds의 price 최대값

In [32]:
df = diamonds.iloc[int(len(diamonds)*0.8):,:]
df.loc[(df['cut'] == 'Fair') & (df['carat']>=1),"price"].max()

2745

#### Airquality
- 8월 21일의 Ozone 값을 구하기

In [38]:
airquality.loc[(airquality['Month'] == 8) & (airquality['Day'] == 21), "Ozone"]

112    21.0
Name: Ozone, dtype: float64

- Ozone 결측값을 없애고 평균 값을 구한 값으로 대체
- median 값에서 2*IQR을 뺀 값과 median에서 2*IQR 을 더한 값 사이에 존재하는 Ozone 값의 합계 구하기

In [9]:
mean = airquality.loc[:,'Ozone'].mean()

In [10]:
airquality_new = airquality.fillna(mean)

In [11]:
IQR = airquality_new['Ozone'].quantile(q=0.75) - airquality_new['Ozone'].quantile(q=0.25)

In [15]:
airquality_new.loc[(airquality_new['Ozone'] <= ub) & (airquality_new['Ozone'] >= lb), 'Ozone'].sum()

4763.7844827586205

In [13]:
lb = airquality_new['Ozone'].median()-2*IQR
ub = airquality_new['Ozone'].median()+2*IQR

#### iris

- Species 가 virginica인 항목에서
- Sepal.Length가 6보다 크면 1, 아니면 0으로 파생 컬럼 Len 을 생성 후, Len의 컬럼 Sum 구하기

In [17]:
iris = pd.read_csv(r'C:\Users\seolbluewings\Desktop\sample\iris.csv')

In [18]:
iris.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [27]:
iris["len"] = np.where(iris.loc[(iris['Species'] == 'Iris-virginica'),].SepalLengthCm > 6, 1,0)

ValueError: Length of values (50) does not match length of index (150)